In [34]:
import pandas as pd
import os
import numpy as np

In [41]:
def feature_adder(filepath, window_size=10):

    df = pd.read_csv(filepath, sep="\s+", header=None, names=["X", "Y"])
    
    df.loc[df["X"] == 0, "X"] = 1e-9


    df['X'] = df['X'] - df['X'].iloc[0]
    df['Y'] = abs(df['Y'] - df['Y'].iloc[0])


    df["ln_t"] = np.log(df["X"])


    df["p_grad"] = np.nan


    for i in range(1, len(df) - 1):
        dp_next = df["Y"].iloc[i+1]
        dp_prev = df["Y"].iloc[i-1]
        ln_t_next = df["ln_t"].iloc[i+1]
        ln_t_prev = df["ln_t"].iloc[i-1]
        
        df.loc[df.index[i], "p_grad"] = (dp_next - dp_prev) / (ln_t_next - ln_t_prev)

    df=df.fillna(0)

    mean_y = df["Y"].mean()
    

    new_cols = {}
    prev_y_cols= []
    next_y_cols = []

    for i in range(1, window_size + 1):
        new_cols[f"prev{i}_y"] = df["Y"].shift(i)

        new_cols[f"next{i}_y"] = df["Y"].shift(-i)
        
        new_cols[f"prev{i}_p_grad"] = df["p_grad"].shift(i)

        new_cols[f"next{i}_p_grad"] = df["p_grad"].shift(-i)
        
        
        prev_y_cols.append(f"prev{i}_y")
        next_y_cols.append(f"next{i}_y")
        

    new_cols_df = pd.DataFrame(new_cols, index=df.index)

    mean_p_grad = df['p_grad'].mean()
    
    for col in new_cols_df.columns:
        if col.endswith("_y"):
            new_cols_df[col].fillna(mean_y, inplace=True)
        elif col.endswith('_p_grad'):
            new_cols_df[col].fillna(mean_p_grad,inplace=True)

    df_result = pd.concat([df, new_cols_df], axis=1)

    
    df_result['delta_P'] = (abs(df_result['Y'] - df_result['prev1_y'])).astype(float)
    df_result.loc[0, 'delta_P'] = 0
    
    
    df_result['avg_y_2n'] = df_result[prev_y_cols + next_y_cols].mean(axis=1)
    df_result['median_y_2n'] = df_result[prev_y_cols + next_y_cols].median(axis=1)
    df_result['avg_y_prevn'] = df_result[prev_y_cols].mean(axis=1)
    df_result['avg_y_nextn'] = df_result[next_y_cols].mean(axis=1)
    df_result['median_y_prevn'] = df_result[prev_y_cols].median(axis=1)
    df_result['median_y_nextn'] = df_result[next_y_cols].median(axis=1)    
    df_result['max-now'] = abs(df_result['Y'] - df_result['Y'].max())
    df_result["diff_from_avg_prev"] = df_result["Y"] - df_result["avg_y_prevn"]
    df_result["diff_from_avg_next"] = df_result["avg_y_nextn"] - df_result["Y"]
    
    
    combined_cols = prev_y_cols + next_y_cols
    
    # Статистические показатели для комбинированного массива:
    df_result['combined_mean'] = df_result[combined_cols].mean(axis=1)
    df_result['combined_median'] = df_result[combined_cols].median(axis=1)
    df_result['combined_std'] = df_result[combined_cols].std(axis=1)
    df_result['combined_skew'] = df_result[combined_cols].skew(axis=1)
    df_result['combined_kurt'] = df_result[combined_cols].kurtosis(axis=1)
    df_result['combined_range'] = df_result[combined_cols].max(axis=1) - df_result[combined_cols].min(axis=1)
    
    # Локальные признаки для поиска КВД/КПД на основе комбинированного массива:
    # Отношение текущего давления к максимальному и минимальному значению среди соседей
    df_result['combined_max'] = df_result[combined_cols].max(axis=1)
    df_result['combined_min'] = df_result[combined_cols].min(axis=1)
    df_result['ratio_to_max_combined'] = df_result['Y'] / df_result['combined_max']
    df_result['ratio_to_min_combined'] = df_result['Y'] / df_result['combined_min']
    
    # Флаг нестабильного интервала:
    # Рассчитываем порог как среднее значение стандартного отклонения для комбинированного массива
    std_threshold = df_result['combined_std'].mean()
    df_result['unstable_flag_combined'] = (df_result['combined_std'] > std_threshold).astype(int)
    
    return df_result

In [42]:
feature_adder('/Users/savinovsvatoslav/Downloads/Task 23/00e03657-8e1e-4c8c-a724-1d3c77b48510').to_csv('proba.csv',index=False)
# feature_adder('/Users/savinovsvatoslav/Code/skvazhina_hack/test')

/Users/savinovsvatoslav/.pyenv/versions/3.8.20/lib/python3.8/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
